In [141]:
import ldclient
from ldclient import Context
from ldclient.config import Config
from faker import Faker
from uuid import uuid4
from faker.providers import DynamicProvider 
import time
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json


In [142]:
sdk_key = 'sdk-c6b1251d-06e7-4efd-9486-2c51570cc5d3'
config = Config(sdk_key = sdk_key, http=ldclient.config.HTTPConfig(connect_timeout=5))
ldclient.set_config(config)
client = ldclient.get()

In [148]:
def parse_string_to_dict(item):
    # Remove surrounding parentheses and split by commas
    clean_item = item.strip("()")
    key_value_pairs = clean_item.split(", ")
    
    # Convert key-value pairs into a dictionary
    parsed_dict = {}
    for pair in key_value_pairs:
        key, value = pair.split("=")  # Split by '='
        parsed_dict[key.strip()] = value.strip()
    
    return parsed_dict

In [143]:
brands_provider  = DynamicProvider(
     provider_name="brand",
     elements=["Admiral", "Diamond", "Elephant", "Toothbrush", "Biscuit"],
)

product_provider = DynamicProvider(
    provider_name="product",
    elements=["Car", "Home", "Motorcycle", "Renters"],
)

price_provider = DynamicProvider(
    provider_name="example_in_cart_price",
    elements=[93.84, 143.73, 101.35, 86.02, 46.91, 125.62, 77.85, 99.68, 73.99, 148.79],
)


tier_provider = DynamicProvider(
    provider_name="tier",
    elements=["Bronze", "Silver", "Gold"],
)



fake = Faker()
fake.add_provider(brands_provider)
fake.add_provider(price_provider)
fake.add_provider(product_provider)
fake.add_provider(tier_provider)

# This is the start of some work on the Percentage Rollout stuff

Change the num_records value to determine how many user contexts to create

In [144]:
num_records = 100
i = 0

fakeUserId2 = []
user2Brand = [] 
user2Price = []
user2Product = []
user2Tier = []

while i <= num_records :
    fakeUserId2.append(fake.uuid4())
    user2Brand.append(fake.brand())
    user2Price.append(fake.example_in_cart_price())
    user2Product.append(fake.product())
    user2Tier.append(fake.tier())
    i = i+1

Now that we have created the records, we want to loop through them and evaluate a feature flag

In [145]:
i = 0
run1 = []
run1_control = []
run1_treatment = []
run1_keys = []

while i <= num_records :

    user2_context = Context.builder(fakeUserId2[i]).kind("user").set("brand", user2Brand[i]).set("product", user2Product[i]).set("tier", user2Tier[i]).build()
    variation_detail = client.variation_detail("example-percentage-rollout-flag", user2_context, 'false')
    run1_keys.append(user2_context.key)
    if (variation_detail.value == "Control"):
        run1_control.append(user2_context.key)
    else:
        run1_treatment.append(user2_context.key)
    time.sleep(0.01)
    run1.append(variation_detail)
    i = i+1

# Here is where you should go and make a change to the percentage rollout

In [146]:
i = 0
run2 = []
run2_control = []
run2_treatment = []
run2_keys = []

while i <= num_records :

    user2_context = Context.builder(fakeUserId2[i]).kind("user").set("brand", user2Brand[i]).set("product", user2Product[i]).set("tier", user2Tier[i]).build()
    variation_detail = client.variation_detail("example-percentage-rollout-flag", user2_context, 'false')
    run2_keys.append(user2_context.key)
    if (variation_detail.value == "Control"):
        run2_control.append(user2_context.key)
    else:
        run2_treatment.append(user2_context.key)
    time.sleep(0.01)
    run2.append(variation_detail)
    i = i+1

How many records have changed variations between runs?

In [147]:
i=0
mismatchCount = 0
while i <= num_records : 
   if (run1[i] != run2[i]):
      mismatchCount+=1
   i+=1   
print(mismatchCount)

23


In [149]:
run1_parsed_data = [parse_string_to_dict(str(item)) for item in run1]
run2_parsed_data = [parse_string_to_dict(str(item)) for item in run2]


In [150]:
run1_df = pd.DataFrame(run1_parsed_data)
run2_df = pd.DataFrame(run2_parsed_data)
keys_df = pd.DataFrame(run1_keys, columns=['key'])
run1_df['key'] = keys_df['key']
run2_df['key'] = keys_df['key']
result = pd.merge(run1_df, run2_df, on='key')

In [152]:
mismatch = result[result['value_x'] != result['value_y']]

In [155]:
mismatch[['key', 'value_x', 'value_y']]

,key,value_x,value_y
0,4aaf5bbb-e4f6-414f-b484-ee2b7bfbd90e,Control,Treatment
3,60712106-7353-4f69-a6eb-39788ef4eb1b,Control,Treatment
8,c3741002-2661-48c2-80d3-0473cbf26ec8,Control,Treatment
9,cb828196-0e4e-4855-b6aa-7f20dec99d44,Control,Treatment
10,e45f9065-d41b-4f0f-bd8b-eac693452311,Control,Treatment
13,109c09e3-e2e4-43f1-98de-b95c5eed1e32,Control,Treatment
20,4ce8d26e-50fe-49e7-ae2f-77c56161fce2,Control,Treatment
25,3f9b8de9-4e46-48fe-abe3-eeaf0e69b188,Control,Treatment
29,2930e6eb-0286-4ec0-8f92-7c7d88740700,Control,Treatment
32,edc49ba7-0e02-4a50-a27c-abb211c9500c,Control,Treatment
